In [1]:
stock = 'MSFT'
days_back = 100

In [2]:
import pandas as pd                              # 表格和数据处理
import matplotlib.pyplot as plt                  # 绘图
import seaborn as sns                            # 更多绘图功能
import pandas_datareader.data as webdata

from itertools import product                    # 一些有用的函数
from tqdm import tqdm_notebook

import datetime                                  # 日期函数

import warnings                                  # 勿扰模式
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
now = datetime.date.today()
start = now - datetime.timedelta(days=days_back)
# df = data.DataReader(stock, "iex", start=start, end=end)
df = webdata.get_data_yahoo(stock, start=start, end=now)['Adj Close']

In [4]:
df

Date
2018-07-23    107.556091
2018-07-24    107.247284
2018-07-25    110.405128
2018-07-26    109.199768
2018-07-27    107.267204
2018-07-30    104.966064
2018-07-31    105.673340
2018-08-01    105.872574
2018-08-02    107.157623
2018-08-03    107.625824
2018-08-06    107.715477
2018-08-07    108.462601
2018-08-08    109.070267
2018-08-09    109.249573
2018-08-10    108.582146
2018-08-13    107.795174
2018-08-14    109.139999
2018-08-15    107.660004
2018-08-16    107.639999
2018-08-17    107.580002
2018-08-20    106.870003
2018-08-21    105.980003
2018-08-22    107.059998
2018-08-23    107.559998
2018-08-24    108.400002
2018-08-27    109.599998
2018-08-28    110.260002
2018-08-29    112.019997
2018-08-30    111.949997
2018-08-31    112.330002
                 ...    
2018-09-18    113.209999
2018-09-19    111.699997
2018-09-20    113.570000
2018-09-21    114.260002
2018-09-24    114.669998
2018-09-25    114.449997
2018-09-26    113.980003
2018-09-27    114.410004
2018-09-28    114.37

In [5]:
# 需要预测的5天日期
d = [None] * 5
j = 0

for i in range(0,7):
    if (now + datetime.timedelta(i)).weekday() < 5:
        d[j] = str(now + datetime.timedelta(i))
        j = j + 1

In [6]:
#时序数据上的线性模型
data = pd.DataFrame(df.copy().astype(float))
data.columns = ["y"]

df = pd.DataFrame({'y':[0,0,0,0,0]},
                   index = d)
data = data.append(df)

for i in range(6, 25):
    data["lag_{}".format(i)] = data.y.shift(i)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

y = data.dropna().y # 删除n/a的数据
X = data.dropna().drop(['y'], axis=1)

# 除去需要预测的5天
train_index = int(len(X)) - 5
X_train = X.iloc[:train_index]
y_train = y.iloc[:train_index]

# 机器学习
lr = LinearRegression()
lr.fit(X_train, y_train)
prediction = lr.predict(X.tail(5))
precentage = prediction.mean()
print(prediction.mean())
print(prediction)
# plt.figure(figsize=(15, 7))
# plt.plot(prediction, "g", label="prediction", linewidth=2.0)


108.7245326512023
[110.01183779 108.42543137 104.06878564 109.07823834 112.03837011]


In [7]:
#储存数据到excel
from openpyxl import load_workbook
df = pd.DataFrame(prediction)
df = df.values.T.tolist()
stock_name = [str(stock)]
wb = load_workbook('/Users/BartSu/Desktop/StockRS_Result.xlsx')
ws = wb.worksheets[0]

ws.append(stock_name)

# Append 2 new Rows - Columns A - D
for row_data in df:
    # Append Row Values
    ws.append(row_data)

wb.save("/Users/BartSu/Desktop/StockRS_Result.xlsx")